In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
# Thank you

StatementMeta(, 1c5062da-8523-4934-84d8-ca794597071d, 5, Submitted, Waiting)

In [1]:
pip install python-dotenv

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 5, Finished, Available)

DEPRECATION: notebookutils 3.4.1-20240110.4 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of notebookutils or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
#upload PR scores to table to AI search index, connect the llm to complete rag
from pyspark.sql import Row, SparkSession
from dotenv import dotenv_values
import openai
from openai import OpenAI
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
import json 
import requests

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 6, Finished, Available)

In [3]:
config = dotenv_values('/lakehouse/default/Files/Credentials.env')

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 7, Finished, Available)

In [4]:
Ai_search_key = config['Ai_search_key']
Ai_search_location = config['Ai_search_location']
Ai_search_endpoint = config['Ai_search_endpoint']
Ai_search_index = 'fabrichackathonindex'
Ai_search_name = 'fabric-hackathon'
Ai_search_index_prscore = 'fabrichackathonindexprscores'

openai_key = config['openai_api_key']
openai_deployment_name = "gpt-4"
openai_url = config['open_ai_endpoint']

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 8, Finished, Available)

In [5]:
def get_embeddings(text):
    client = OpenAI(api_key = openai_key)

    response = client.embeddings.create(
        input = text,
        model= "text-embedding-ada-002"
    )
    embeddings = response.data[0].embedding  
    return embeddings

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 9, Finished, Available)

In [6]:
def gen_category_embedding(RoundType):
    """Generates embedding for each roundtype."""

    df_RoundType = spark.createDataFrame([(RoundType, 1)], ["RoundType", "dummy"])
    # Define a UDF (User Defined Function)
    get_embeddings_udf = udf(get_embeddings, ArrayType(FloatType()))
    # Apply the UDF to the  column
    df_cat_embeddings = df_RoundType.withColumn('embeddings', get_embeddings_udf(df_RoundType['RoundType']))
    row = df_cat_embeddings.collect()[0]
    category_embedding = row.embeddings
    return category_embedding


StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 10, Finished, Available)

In [7]:
def retrieve_top_chunks(k, category, category_embedding):
    """Retrieve the top K entries from Azure AI Search using hybrid search."""
    url = f"https://{Ai_search_name}.search.windows.net/indexes/{Ai_search_index}/docs/search?api-version=2023-11-01"

    payload = json.dumps({
        "search": category,
        "top": k,
        "vectorQueries": [
            {
                "vector": category_embedding,
                "k": k,
                "fields": "contentVector",
                "kind": "vector"
            }
        ]
    })

    headers = {
        "Content-Type": "application/json",
        "api-key": Ai_search_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    output = json.loads(response.text)
    return output



StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 11, Finished, Available)

In [8]:
def retrieve_top_chunks_prscore(k, category, category_embedding):
    """Retrieve the top K entries from Azure AI Search using hybrid search."""
    url = f"https://{Ai_search_name}.search.windows.net/indexes/{Ai_search_index_prscore}/docs/search?api-version=2023-11-01"

    payload = json.dumps({
        "search": category,
        "top": k,
        "vectorQueries": [
            {
                "vector": category_embedding,
                "k": k,
                "fields": "embeddings",
                "kind": "vector"
            }
        ]
    })

    headers = {
        "Content-Type": "application/json",
        "api-key": Ai_search_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    output = json.loads(response.text)
    return output



StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 12, Finished, Available)

In [9]:
def get_context(category, retrieved_k = 5):
    # Generate embeddings for the categories
    category_embedding = gen_category_embedding(category)

    # Retrieve the top K entries
    output = retrieve_top_chunks(retrieved_k, category, category_embedding)

    # concatenate the content of the retrieved documents
    context = [chunk["Article_body"] for chunk in output["value"]]

    return context

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 13, Finished, Available)

In [10]:
def get_contextprscore(category, retrieved_k = 5):
    # Generate embeddings for the categories
    category_embedding = gen_category_embedding(category)

    # Retrieve the top K entries
    output = retrieve_top_chunks_prscore(retrieved_k, category, category_embedding)

    # concatenate the content of the retrieved documents
    url = output['value'][0]["URL_of_Draw_Details"]
    Round_Type = output['value'][0]["Round_Type"]
    Draw_Date = output['value'][0]["Draw_Date"]
    Invitations_Issued = output['value'][0]["Invitations_Issued"]
    crs_score = output['value'][0]["CRS_Cutoff_Score"]

    output = f' Class is {Round_Type}:, cutoff score is {crs_score}, date of draw is {Draw_Date} and number of invites is {Invitations_Issued}. url is {url}'
    return output

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 14, Finished, Available)

In [ ]:
context = get_context("what are the canadian PR classes", retrieved_k = 5)

StatementMeta(, , , Waiting, )

In [ ]:
context

StatementMeta(, , , Waiting, )

In [11]:
df = spark.sql("SELECT * FROM CanadianPRScores.CanadaPRScores LIMIT 1000")
display(df)

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, 330216ce-4c19-417a-8bd7-85f76b3f4006)

In [12]:
def prompt_engineering(context,score):
    article = context[0]
    score = score
    chat_context_prompt = f"""

    You are an assistant to answer queries about the Canadian Permanent Residence. 
    Do not hallucinate. 
    Use information from either the article or score below to answer your questions
    
    article: {article}

    score : {score}

    If the answer to the query is not in article or score above,respond 'I am unable to provide a response on that'

    """

    return chat_context_prompt


StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 16, Finished, Available)

In [13]:
client = OpenAI(api_key=openai_key)

def PR_Assistant(text):
    MESSAGES = [
    {"role": "system", "content": chat_context_prompt},
    {"role": "user", "content": text},
    ]
    MESSAGES.append({"role": "user", "content": text})

    completion = client.chat.completions.create(model="gpt-4", messages=MESSAGES,temperature=0.9)
    return completion.choices[0].message.content




StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 17, Finished, Available)

In [14]:
question = "What are the canadian permanent resident classes?"

context = get_context(question, retrieved_k = 5)

score = get_contextprscore(question, retrieved_k = 1)

chat_context_prompt = prompt_engineering(context, score)

client_response = PR_Assistant(question)

print(client_response)

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 18, Finished, Available)

The Canadian permanent resident classes include:

1. Federal skilled workers (FSW)
2. Canadian experience class (CEC)
3. Federal skilled trades (FST)
4. Start-up business class
5. Investors (Closed on December 30, 2015)
6. Entrepreneurs and self-employed persons
7. Self-employed persons class
8. Quebec Economic Classes
9. Provincial nominees
10. Atlantic Immigration Pilot Programs (AIPP) (Closed on March 5, 2022)
11. Atlantic Immigration Program (AIP)
12. Caregiver classes
13. Rural and Northern Immigration Pilot Program
14. Agri-Food Pilot (AFP)
15. Temporary Resident to Permanent Resident Pathway
16. Economic Mobility Pathways Pilot (EMPP)


In [15]:
question = "What is the most drawn class"

context = get_context(question, retrieved_k = 5)

score = get_contextprscore(question, retrieved_k = 1)

chat_context_prompt = prompt_engineering(context, score)

client_response = PR_Assistant(question)

print(client_response)

StatementMeta(, 29b01f49-ce4b-4f91-84e8-620daaf0b10e, 19, Finished, Available)

The provided information does not specify which class is the most drawn for Canadian Permanent Residence.
